In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa

In [2]:
# gpu_devices = tf.config.experimental.list_physical_devices('GPU')
# print("Num GPUs Available: ", len(gpu_devices))
# for device in gpu_devices:
#     tf.config.experimental.set_memory_growth(device, True)

In [3]:
from bert.model import create_albert_model
model = create_albert_model(model_dimension=1024,
                            transformer_dimension=1024 * 4,
                            num_attention_heads=1024 // 64,
                            num_transformer_layers=24,
                            vocab_size=22,
                            dropout_rate=0.)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
token_embedding (TokenEmbedding (None, None, 1024)   22528       input_1[0][0]                    
__________________________________________________________________________________________________
transformer (Transformer)       (None, None, 1024)   12598912    token_embedding[0][0]            
                                                                 transformer[0][0]                
                                                                 transformer[1][0]                
                                                                 transformer[2][0]            

In [4]:
from bert.optimizers import (ECE, masked_sparse_categorical_crossentropy,
                             BertLinearSchedule)

opt = tfa.optimizers.AdamW(learning_rate=1E-4,
                           beta_2=0.98,
                           epsilon=1E-6,
                           weight_decay=0.001)

opt = tf.train.experimental.enable_mixed_precision_graph_rewrite(opt)

true_labels = tf.keras.layers.Input(
    shape=(None,), dtype=tf.int32, batch_size=None)

model.compile(
    target_tensors=true_labels,    
    loss=masked_sparse_categorical_crossentropy,
    metrics=[ECE],
    optimizer=opt,
    experimental_run_tf_function=True)

In [5]:
callbacks = [    
    BertLinearSchedule(1E-4, 10000, int(1E7)),
    tf.keras.callbacks.ModelCheckpoint(filepath='/scratch/pstjohn/test_ckpts/ckpt_{epoch}_{val_ECE:.2f}.h5'),
]

In [6]:
from bert.dataset import create_masked_input_dataset

training_data = create_masked_input_dataset(
    sequence_path='../uniparc_data/sequences_train.txt',
    max_sequence_length=768,
    masking_freq=0.05,    
    batch_size=4)

training_data = training_data.repeat().prefetch(tf.data.experimental.AUTOTUNE)

valid_data = create_masked_input_dataset(
    sequence_path='../uniparc_data/sequences_valid.txt',
    max_sequence_length=768,
    masking_freq=0.05,
    batch_size=4)

valid_data = valid_data.repeat().prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
model.fit(training_data, steps_per_epoch=10000, epochs=10,
          verbose=1, validation_data=valid_data, validation_steps=10,
          callbacks=callbacks)

Epoch 1/10


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


  371/10000 [>.............................] - ETA: 3:11:03 - loss: 2.9489 - ECE: 19.2507